# ODESystem Class Demonstration

This notebook provides a comprehensive demonstration of the `ODESystem` class and all of its methods. The ODESystem class implements the ordinary differential equation system for compartmental neuronal modeling using the Hodgkin-Huxley formalism.

## Overview

The ODESystem class:
- Takes a `SegmentGraph` as input representing the neuronal compartments
- Creates a system of ODEs modeling membrane potential dynamics
- Implements Hodgkin-Huxley ionic currents (Na+, K+, leak)
- Supports multiple numerical integration methods (Forward Euler, SciPy solvers)
- Handles stimulus protocols and parameter customization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from typing import Dict, List, Tuple

# Import GenCoMo modules
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

from gencomo.ode import ODESystem
from gencomo.segmentation import SegmentGraph

print("📦 Imports successful!")

## 1. Creating a Simple SegmentGraph

First, let's create a simple SegmentGraph to work with. We'll create a linear chain of compartments to demonstrate the ODESystem functionality.

In [ ]:
def create_demo_segment_graph(num_compartments: int = 3) -> SegmentGraph:
    """
    Create a demo SegmentGraph with a linear chain of compartments.
    """
    graph = SegmentGraph()
    
    # Add compartments as nodes
    for i in range(num_compartments):
        comp_id = f"comp_{i}"
        
        # Add node with properties
        graph.add_node(comp_id, 
                      external_surface_area=10.0 + i * 2.0,  # μm²
                      volume=5.0 + i,  # μm³
                      membrane_potential=-65.0,  # mV
                      slice_index=i,
                      centroid=np.array([0.0, 0.0, i * 10.0]))
    
    # Add connections between adjacent compartments
    for i in range(num_compartments - 1):
        comp1 = f"comp_{i}"
        comp2 = f"comp_{i+1}"
        
        # Add edge with conductance
        graph.add_edge(comp1, comp2, 
                      conductance=1e-6,  # S (siemens)
                      length=10.0)  # μm
    
    return graph

# Create demo graph
demo_graph = create_demo_segment_graph(4)

print(f"🧠 Created demo SegmentGraph:")
print(f"   Nodes: {list(demo_graph.nodes())}")
print(f"   Edges: {list(demo_graph.edges())}")

# Visualize the graph structure
plt.figure(figsize=(10, 6))
pos = nx.spring_layout(demo_graph)
nx.draw(demo_graph, pos, with_labels=True, node_color='lightblue', 
        node_size=1000, font_size=10, font_weight='bold')
plt.title("Demo SegmentGraph Structure")
plt.show()

## 2. ODESystem Initialization and Basic Properties

Let's create an ODESystem instance and explore its basic properties and helper methods.

In [ ]:
# Create ODESystem instance
ode_system = ODESystem(demo_graph)

print("🔧 ODESystem created!")
print(f"String representation: {ode_system}")
print(f"Repr: {repr(ode_system)}")
print()

# Print detailed summary
ode_system.print_summary()

## 3. Exploring Compartment Information

The ODESystem provides methods to inspect individual compartments and their properties.

In [ ]:
# Get information about each compartment
print("📊 Compartment Information:")
print("=" * 50)

for comp_id in ode_system.compartment_ids:
    info = ode_system.get_compartment_info(comp_id)
    print(f"\n🔹 {comp_id}:")
    print(f"   Index: {info['index']}")
    print(f"   Surface area: {info['external_surface_area']:.1f} μm²")
    print(f"   Volume: {info['volume']:.1f} μm³")
    print(f"   Membrane potential: {info['membrane_potential']:.1f} mV")
    print(f"   Neighbors: {info['neighbors']} ({info['num_neighbors']} total)")
    print(f"   Centroid: {info['centroid']}")

## 4. Parameter Management

The ODESystem allows customization of biophysical parameters.

In [ ]:
# Show default parameters
print("⚙️ Default Biophysical Parameters:")
for param, value in ode_system.default_params.items():
    print(f"   {param}: {value}")

print("\n🔧 Modifying parameters...")

# Modify some parameters
ode_system.set_parameters(
    temperature=310.15,  # 37°C in Kelvin
    na_conductance=0.15,  # Increase sodium conductance
    k_conductance=0.04   # Increase potassium conductance
)

print("\n📊 Updated Parameters:")
for param, value in ode_system.default_params.items():
    print(f"   {param}: {value}")

## 5. Stimulus Protocols

The ODESystem supports adding various stimulus protocols to compartments.

In [ ]:
# Add stimuli to different compartments
print("⚡ Adding stimulus protocols...")

# Current injection stimulus
ode_system.add_stimulus(
    compartment_id="comp_0",
    start_time=10.0,  # ms
    duration=5.0,     # ms
    amplitude=50.0,   # pA
    stimulus_type="current"
)

# Voltage clamp stimulus
ode_system.add_stimulus(
    compartment_id="comp_2",
    start_time=20.0,  # ms
    duration=3.0,     # ms
    amplitude=-30.0,  # mV
    stimulus_type="voltage"
)

# Print updated summary with stimuli
ode_system.print_summary()